## FILTRO PER FAKE NEWS 🔎🔎

Il problema delle fake news è cresciuto esponenzialmente nell'ultimo decennio a causa della crescente diffusione dei social network, il governo degli Stati Uniti ha deciso di muoversi a tal proposito, incaricando la tua azienda di realizzare un plug-in per chrome in grado di riconoscere se una notizia è falsa. 

Il tuo compito è quello di realizzare il modello in grado di riconoscere le notizie false, che poi il team di machine learning enginner e web developer metterà in produzione.

Ti vengono messi a disposizioni due raccolte di notizie, una contenente solo notizie false e l'altra contenente solo notizie vere, utilizzale per addestrare il tuo modello.


Parti da un'accurata analisi, rispondendo a domande come:

* le fake news sono più frequenti in una determinata categoria?
* per ogni categoria, ci sono argomenti che sono più soggetti alle fake news?
* I titoli delle fake news presentano dei pattern?

Una volta addestrato il modello esportalo utilizzando pickle così che i tuoi colleghi possano metterlo in produzione.

In [ ]:
!pip install pandas spacy nltk
!python -m spacy download en_core_web_sm
import nltk
nltk.download('stopwords')

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score,confusion_matrix, precision_score, recall_score, f1_score
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import OneHotEncoder, FunctionTransformer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression

import spacy
import string
import re
from nltk.corpus import stopwords

from collections import Counter
from sklearn.feature_extraction.text import CountVectorizer
import gensim
from gensim.utils import simple_preprocess as sp
from gensim import corpora
from gensim.models import LdaMulticore
from pprint import pprint

nlp = spacy.load('en_core_web_sm')
english_stopwords = set(stopwords.words('english'))

In [ ]:
df_true = pd.read_csv("True.csv")
df_true.head()

In [ ]:
df_true.info()

In [ ]:
df_fake = pd.read_csv("Fake.csv")
df_fake.head()

In [ ]:
df_fake.info()

In [ ]:
true_subjects = df_true['subject'].unique()
fake_subjects = df_fake['subject'].unique()

print("True:", true_subjects)
print("Fake:", fake_subjects)

### Data Preprocessing

In [ ]:
def clean_title(sentence):
    sentence = sentence.lower()
    for i in string.punctuation:
        sentence = sentence.replace(i, " ")
    
    document = nlp(sentence)
    sentence = " ".join(token.lemma_ for token in document)
    sentence = " ".join(word for word in sentence.split() if word not in english_stopwords)
    sentence = re.sub("\d", "", sentence)
    sentence = re.sub(" +", " ", sentence)
    
    return sentence

df_true['title'] = df_true['title'].apply(clean_title)
df_fake['title'] = df_fake['title'].apply(clean_title)

In [ ]:
#ho creato una funzione più semplice, 
#poiché applicare la funzione clean_title() anche sul testo portava via troppo tempo
def clean_text(text):
    text = text.lower()
    text = re.sub(r'\d+', '', text) 
    text = re.sub(r'[^\w\s]', '', text)
    text = " ".join([word for word in text.split() if word not in english_stopwords])
    return text

df_true['text'] = df_true['text'].apply(clean_text)
df_fake['text'] = df_fake['text'].apply(clean_text)

In [ ]:
df_true['title'] = df_true['title'].astype('string')
df_true['text'] = df_true['text'].astype('string')
df_fake['title'] = df_fake['title'].astype('string')
df_fake['text'] = df_fake['text'].astype('string')

In [ ]:
df_true['subject'] = df_true['subject'].astype('category')
df_fake['subject'] = df_fake['subject'].astype('category')

In [ ]:
df_true['date'] = df_true['date'].str.strip()
df_true['date'] = pd.to_datetime(df_true['date'], errors='coerce')
df_fake['date'] = df_fake['date'].str.strip()
df_fake['date'] = pd.to_datetime(df_fake['date'], errors='coerce')
print(df_true['date'].isna().sum())
print(df_fake['date'].isna().sum())

### Le fake news sono più frequenti in una determinata categoria?

In [ ]:
subject_counts = df_fake['subject'].value_counts()

plt.figure(figsize=(10, 6))
sns.barplot(x=subject_counts.index, y=subject_counts.values, palette='viridis')
plt.title('Frequenza di ciascun Subject')
plt.xlabel('Subject')
plt.ylabel('Frequenza')
plt.xticks(rotation=45)
plt.show()

### Per ogni categoria, ci sono argomenti che sono più soggetti alle fake news?

In [ ]:
# Funzioni di Preprocessamento
def sent_to_words(items):
    for item in items:
        yield(sp(item, deacc=True))
        

subjects = df_fake['subject'].unique()

# Creazione del modello LDA per ciascun subset
topic_models = {}
for subject in subjects:
    print(f"Processing subject: {subject}")
    # Filtra il dataframe per il subject corrente
    subset = df_fake[df_fake['subject'] == subject]
    documents = subset['text']
    
    # Preprocessing del testo
    data_words = list(sent_to_words(documents))
    
    # Creazione del dizionario e del corpus
    id2word = corpora.Dictionary(data_words)
    corpus = [id2word.doc2bow(text) for text in data_words]
    
    # Creazione e addestramento del modello LDA
    lda_model = LdaMulticore(corpus=corpus, id2word=id2word, num_topics=3, passes=3)
    topic_models[subject] = lda_model
    pprint(lda_model.print_topics())


Considerazioni Finali

- Omogeneità Tematica: La forte omogeneità dei temi tra diverse categorie potrebbe indicare che le fake news tendono a sfruttare argomenti politicamente e socialmente sensibili che hanno un impatto trasversale, indipendentemente dalla specifica categoria di notizia.


- Polarizzazione e Manipolazione: La presenza dominante di figure politiche chiave e l'uso di temi vaghi suggeriscono che le fake news sono costruite per polarizzare l'opinione pubblica e manipolare le emozioni, piuttosto che informare con contenuti specifici e dettagliati.


- Raffinamento dell'Analisi: Per un'analisi più approfondita, potrebbe essere utile confrontare questi risultati con quelli delle notizie vere nelle stesse categorie. Questo confronto potrebbe rivelare se e come le fake news si differenziano dalle notizie vere in termini di focus tematico.


### I titoli delle fake news presentano dei pattern?

In [ ]:
# Utilizzare CountVectorizer per contare le frequenze delle parole
vectorizer = CountVectorizer(stop_words='english')
title_matrix = vectorizer.fit_transform(df_fake['title'])
sum_words = title_matrix.sum(axis=0)
words_freq = [(word, sum_words[0, idx]) for word, idx in vectorizer.vocabulary_.items()]
words_freq = sorted(words_freq, key=lambda x: x[1], reverse=True)

# Visualizzare le prime 10 parole più frequenti
print(words_freq[:10])

Commento:

La prevalenza di nomi di personaggi politici indica che una parte significativa delle fake news è centrata su figure politiche di spicco, in particolare legate alla politica statunitense. Questo suggerisce che le fake news tendono a concentrarsi su argomenti politicamente polarizzanti, usando figure controverse per attirare l'attenzione.

Il risultato conferma che i titoli delle fake news presentano pattern riconoscibili, con un forte orientamento verso la politica e l'uso di linguaggio sensazionalistico. Questi pattern non solo riflettono i temi che probabilmente attraggono l'attenzione del pubblico, ma anche le tecniche utilizzate per massimizzare l'engagement, come l'uso di termini specifici che suggeriscono urgenza o esclusività.

### creare il modello di classificazione(logistic regression)

In [ ]:
df_true['truthful'] = 1
df_fake["truthful"] = 0

df = pd.concat([df_true, df_fake], ignore_index=True)
#df.drop(["date"], axis = 1, inplace = True)
df = df.sample(frac=1).reset_index(drop=True)
df['subject'] = df['subject'].astype('category')

# Calcolo della data mediana
median_date = df['date'].median()
# Imputazione dei valori mancanti con la data mediana
df['date'].fillna(median_date, inplace=True)

In [ ]:
df.info()

In [ ]:
df.head()

In [ ]:
# Funzione per estrarre componenti dalla data
def extract_date_features(X):
    X['year'] = X['date'].dt.year
    X['month'] = X['date'].dt.month
    X['day_of_week'] = X['date'].dt.dayofweek
    return X[['year', 'month', 'day_of_week']]

X = df[['title', 'text', 'subject', 'date']]
y = df['truthful']
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.4, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

preprocessor = ColumnTransformer(
    transformers=[
        ('bow_title', CountVectorizer(stop_words='english'), 'title'),
        ('bow_text', CountVectorizer(stop_words='english'), 'text'),
        ('cat', OneHotEncoder(), ['subject']),
        ('date_features', FunctionTransformer(extract_date_features), ['date'])
    ])

pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('classifier', LogisticRegression())
])

# Addestramento del modello sul training set
pipeline.fit(X_train, y_train)

# Valutazione del modello sul validation set
y_val_pred = pipeline.predict(X_val)

# Valutazione delle prestazioni del modello sul validation set
accuracy = accuracy_score(y_val, y_val_pred)
precision = precision_score(y_val, y_val_pred)
recall = recall_score(y_val, y_val_pred)
f1 = f1_score(y_val, y_val_pred)
print(f'Accuracy: {accuracy:.2f}')
print(f'Precision: {precision:.2f}')
print(f'Recall: {recall:.2f}')
print(f'F1 Score: {f1:.2f}')

# Matrice di confusione sul validation set
cm_val = confusion_matrix(y_val, y_val_pred)

# Visualizzazione della matrice di confusione come grafico
plt.figure(figsize=(8, 6))
sns.heatmap(cm_val, annot=True, fmt='d', cmap='Blues', xticklabels=['Fake', 'True'], yticklabels=['Fake', 'True'])
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.title('Confusion Matrix for Validation Set')
plt.show()


In [ ]:
# Valutazione finale sul test set
y_test_pred = pipeline.predict(X_test)

# Valutazione delle prestazioni del modello sul test set
test_accuracy = accuracy_score(y_test, y_test_pred)
test_precision = precision_score(y_test, y_test_pred)
test_recall = recall_score(y_test, y_test_pred)
test_f1 = f1_score(y_test, y_test_pred)
print(f'Test Set Metrics:')
print(f'Accuracy: {test_accuracy:.2f}')
print(f'Precision: {test_precision:.2f}')
print(f'Recall: {test_recall:.2f}')
print(f'F1 Score: {test_f1:.2f}')

# Matrice di confusione sul test set
cm_test = confusion_matrix(y_test, y_test_pred)

# Visualizzazione della matrice di confusione come grafico per il test set
plt.figure(figsize=(8, 6))
sns.heatmap(cm_test, annot=True, fmt='d', cmap='Greens', xticklabels=['Fake', 'True'], yticklabels=['Fake', 'True'])
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.title('Confusion Matrix for Test Set')
plt.show()

NOTA:

Abbiamo applicato una trasformazione CountVectorizer sulle colonne testuali, ma ci sono anche altre trasformazioni possibili:
- TfidfVectorizer
- Word Embeddings
- N-grams
- HashingVectorizer
- Feature Engineering con Meta-dati
- Dimensionality Reduction